In [15]:
import os
from langchain_asi import ASI1ChatModel
import pdfplumber


In [16]:
asi1_model = ASI1ChatModel(
    model="asi1-mini",
    api_key=os.getenv("ASI1_API_KEY"),
)

In [17]:
def leer_texto_pdf(path_pdf):
    with pdfplumber.open(path_pdf) as pdf:
        texto = "\n".join([pagina.extract_text() for pagina in pdf.pages if pagina.extract_text()])
    return texto

In [18]:
def construir_prompt(texto_factura):
    prompt = f"""
Extrae los siguientes campos de esta factura:

- Nombre del emisor
- RFC del emisor
- Fecha de emisión
- Nombre del receptor
- RFC del receptor
- Moneda
- Subtotal
- IVA
- Total

Factura:
---
{texto_factura}
---
Responde solo en formato JSON válido y sin ningún comentario adicional.
    """
    return prompt

In [19]:
def extraer_datos_con_asi1(path_pdf):
    texto_factura = leer_texto_pdf(path_pdf)
    prompt = construir_prompt(texto_factura)
    respuesta = asi1_model.invoke(prompt)
    
    # Obtener el contenido de la respuesta
    contenido = respuesta.content
    
    # Intentar diferentes formatos de extracción
    try:
        # Si el contenido ya es un JSON válido
        if contenido.strip().startswith('{'):
            return contenido
        
        # Si está envuelto en markdown
        if '```json' in contenido:
            json_str = contenido.split('```json\n')[1].split('\n```')[0]
            return json_str
        
        # Si está envuelto en markdown sin especificar el lenguaje
        if '```' in contenido:
            json_str = contenido.split('```\n')[1].split('\n```')[0]
            return json_str
            
        # Si no se encuentra ningún formato específico, devolver el contenido tal cual
        return contenido
        
    except Exception as e:
        print(f"Error al procesar la respuesta: {e}")
        print("Contenido original:", contenido)
        return contenido


In [20]:
resultado = extraer_datos_con_asi1(r"C:\Users\josei\fr8-projects\GitHub\fr8-invoices-information\data\raw\unique\Estadía 53572-1.pdf")
print(resultado)


CropBox missing from /Page, defaulting to MediaBox
CropBox missing from /Page, defaulting to MediaBox


{
  "Nombre del emisor": "ITZEL ESTEFANIA HERNANDEZ RAMIREZ",
  "RFC del emisor": "HERI910707B41",
  "Fecha de emisión": "2024-02-14T08:58:41",
  "Nombre del receptor": "FREIGHT APP DE MEXICO",
  "RFC del receptor": "FHM190118EN7",
  "Subtotal": 7800.00,
  "IVA": 1248.00,
  "Total": 9048.00
}
